kudos @hamditarek https://www.kaggle.com/hamditarek/market-prediction-xgboost-with-gpu-fit-in-1min

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

#################################################################

In [ ]:
import xgboost as xgb
print("XGBoost version:", xgb.__version__)

In [ ]:
%%time

import cudf

train_cudf  = cudf.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
train = train_cudf.to_pandas()
del train_cudf

features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')

print ("Data is loaded!")

In [ ]:
print('train shape is {}'.format(train.shape))
print('features shape is {}'.format(features.shape))
print('example_test shape is {}'.format(example_test.shape))
print('sample_prediction_df shape is {}'.format(sample_prediction_df.shape))

In [ ]:
missing_values_count = train.isnull().sum()
print (missing_values_count)

total_cells = np.product(train.shape)
total_missing = missing_values_count.sum()
print ("\n Percentage of missing data = ",(total_missing/total_cells) * 100,"%")

In [ ]:
train = train[train['weight'] != 0]
print(train.shape)
print(train.head())
#making resp as a target
train['action'] = (train['resp'].values > 0).astype('int')
print(train.shape)
print(train.head())

X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']

In [ ]:
f_mean = X_train.mean()
X_train.fillna(f_mean)

In [ ]:
# we need to fine tune using hyperplot or randomgridsearch

clf = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=7,
    eta=0.5, 
    missing=None,
    random_state=42,
    tree_method='gpu_hist', # to make xgb use gpu
    subsample=0.8,
    colsample_bytree=1,
    verbosity=2   
)

In [ ]:
%%time
clf.fit(X_train, y_train)

##################################################################

In [ ]:
%%time
# iter_test give as the test data to predict
for (test_df, sample_prediction_df) in iter_test:
    
    # get the data from iter_test
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    y_preds = clf.predict(X_test)
    
    # test the data through janestreet api
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)